In [ ]:
from dataset import get_dataloader
import matplotlib.pyplot as plt

In [ ]:
dataloader = get_dataloader(1)

In [ ]:
stop = 5
cnt = 0
for i, (waveform, lengths) in enumerate(dataloader):
    # plot the waveform
    plt.plot(waveform[0].numpy())
    plt.show()
    cnt += 1
    if cnt == stop:
        break

In [1]:
from dinosr_config import DinosrAudioConfig

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = DinosrAudioConfig(
        average_top_k_layers=6,
        encoder_layers=8,
        encoder_embed_dim=256,
        model_type="transformer",
)
cfg.save("meow.yaml")

cfg2 = DinosrAudioConfig.load("meow.yaml")